In [28]:
import os
import numpy as np
import pandas as pd
#from matplotlib import pyplot as plt
import numpy as np  
#from skimage.metrics import structural_similarity
#from sklearn.cluster import KMeans
import random
#import openslide
import copy

#import matplotlib.image

In [ ]:
# Code for NRB

In [29]:
def cossim(Q, K):
    K_norm = np.linalg.norm(K, axis=1, keepdims=True)
    Q_norm = np.linalg.norm(Q, axis=1, keepdims=True)
    QK_norm = Q_norm @ K_norm.T
    QK = Q @ K.T

    return QK / QK_norm

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

### Normal

In [4]:
className = "normal"

for threshold in [0.95]:
    baseDir = "./CM16/Ziyu/feats/train/{}/".format(className)
    tumorLabelsPath = "./CM16/Ziyu/locs/labels/{}labels.npy"
    locs = "./CM16/Ziyu/locs/{}"

    #threshold = 0.95
    labelsArr = {}

    for WSI in os.listdir(baseDir):
        NRB = np.zeros([1, 1024])
        tumorLabels = []
        locPts = []

        bag = np.load(baseDir + WSI)
        WSILabels = np.load(tumorLabelsPath.format(WSI.split(".")[0]))
        locsPoints = np.load(locs.format(wsi))
        
        while(len(bag) != 0):
            randomNo = random.randint(0, len(bag) - 1)
            tumorLabels.append(WSILabels[randomNo])
            locPts.append(locsPoints[randomNo])
            randomFE = bag[randomNo, :].reshape(1, -1)

            bag = np.delete(bag, randomNo, 0)
            WSILabels = np.delete(WSILabels, randomNo, 0)

            NRB = np.append(NRB, randomFE, axis = 0)

            if (len(bag) > 0):
                res = cossim(randomFE, bag)
                maxSim = np.max(res)

                if (maxSim > threshold):
                    bag = np.delete(bag, np.argwhere(res > threshold)[:, 1], 0)
                    WSILabels = np.delete(WSILabels, np.argwhere(res > threshold)[:, 1], 0)

        np.save("./CM16/Ziyu/feats/train/{}_randomalgo_{}/{}".format(className, threshold, WSI), NRB[1:, :])
        if (className != "normal"):
            np.save("./CM16/Ziyu/feats/train/{}_randomalgo_{}/{}labels.npy".format(className, threshold, WSI.split(".")[0]), tumorLabels)

        labelsArr[WSI] = sum(tumorLabels)
        #print(WSI, "Actual :", bag.shape[0])
        #print(sum(tumorLabels), ", ", len(tumorLabels))

FileNotFoundError: [Errno 2] No such file or directory: './CM16/Ziyu/locs/labels/normal_156labels.npy'

In [3]:
def computeRepresentativeVectors(WSI, tumorLabels, locPts, threshold):    
    NRB = np.zeros([1, 1024])
    #tumorLabels = []
    #locPts = []
    
    bag = np.load(WSI)
    
    wsiName = WSI.split("/")[-1].split(".")[0]
    
    #WSILabels = np.load(tumorLabelsPath.format(wsiName))
    #locsPoints = np.load("./CM16/Ziyu/locs/{}.npy".format(wsiName))
    
    while(len(bag) != 0):
        randomNo = random.randint(0, len(bag) - 1)
        tumorLabels.append(WSILabels[randomNo])
        locPts.append(locsPoints[randomNo])
        
        randomFE = bag[randomNo, :].reshape(1, -1)

        bag = np.delete(bag, randomNo, 0)
        WSILabels = np.delete(WSILabels, randomNo, 0)
        
        NRB = np.append(NRB, randomFE, axis = 0)

        if (len(bag) > 0):
            res = cossim(randomFE, bag)
            maxSim = np.max(res)

            if (maxSim > threshold):
                bag = np.delete(bag, np.argwhere(res > threshold)[:, 1], 0)
                WSILabels = np.delete(WSILabels, np.argwhere(res > threshold)[:, 1], 0)
                locsPoints = np.delete(locsPoints, np.argwhere(res > threshold)[:, 1], 0)
                
    return NRB[1:, :], tumorLabels, locPts

In [ ]:
### Creation of NRB using 0.95 tumor threshold

In [32]:
className = "normal"
baseDir = "./CM16/Ziyu/feats/train/{}/".format(className)

for threshold in [0.95]:
    tumorLabelsPath = "./CM16/Ziyu/locs/labels/{}labels.npy"

    labelsArr = {}

    for WSI in os.listdir(baseDir):
        print(WSI)
        NRB, tumorLabels, locs = computeRepresentativeVectors(baseDir + WSI, tumorLabelsPath, threshold)
        
        np.save("./CM16/Ziyu/feats_2_{}/train/{}/{}".format(threshold, className, WSI), NRB)
        if (className != "normal"):
            np.save("./CM16/Ziyu/feats_2_{}/train/{}/{}labels.npy".format(threshold, className, WSI.split(".")[0]), tumorLabels)
        np.save("./CM16/Ziyu/feats_2_{}/locs/{}".format(threshold, WSI), locs)
        
        labelsArr[WSI] = sum(tumorLabels)

normal_156.npy
normal_141.npy
normal_053.npy
normal_082.npy
normal_142.npy
normal_021.npy
normal_152.npy
normal_067.npy
normal_004.npy
normal_136.npy
normal_014.npy
normal_137.npy
normal_092.npy
normal_110.npy
normal_130.npy
normal_100.npy
normal_029.npy
normal_153.npy
normal_088.npy
normal_076.npy
normal_056.npy
normal_122.npy
normal_039.npy
normal_066.npy
normal_128.npy
normal_063.npy
normal_081.npy
normal_026.npy
normal_064.npy
normal_034.npy
normal_030.npy
normal_093.npy
normal_144.npy
normal_155.npy
normal_043.npy
normal_062.npy
normal_095.npy
normal_132.npy
normal_089.npy
normal_012.npy
normal_138.npy
normal_158.npy
normal_135.npy
normal_061.npy
normal_020.npy
normal_115.npy
normal_058.npy
normal_057.npy
normal_154.npy
normal_131.npy
normal_049.npy
normal_023.npy
normal_003.npy
normal_114.npy
normal_018.npy
normal_140.npy
normal_013.npy
normal_150.npy
normal_118.npy
normal_065.npy
normal_151.npy
normal_116.npy
normal_160.npy
normal_068.npy
normal_113.npy
normal_040.npy
normal_149